In [108]:
%matplotlib inline
import glob
from sklearn.naive_bayes import MultinomialNB
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from nltk import sent_tokenize
from nltk import pos_tag
from nltk import map_tag
from sklearn.metrics import classification_report
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [96]:
# just to see the non null objects.
icse = pd.read_csv("icse_id.txt", delimiter='\t')
icse.columns = ['id','title','title_small','year','date','unnamed','conference','conference_short','reference1','reference2','reference3']
icse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10085 entries, 0 to 10084
Data columns (total 11 columns):
id                  10085 non-null object
title               10085 non-null object
title_small         10085 non-null object
year                10085 non-null int64
date                9673 non-null object
unnamed             6636 non-null object
conference          10085 non-null object
conference_short    10085 non-null object
reference1          15 non-null object
reference2          10085 non-null object
reference3          10085 non-null int64
dtypes: int64(2), object(9)
memory usage: 866.8+ KB


In [97]:
sigmod = pd.read_csv("sigmod_id.txt", delimiter='\t')
sigmod.columns = ['id','title','title_small','year','date','unnamed','conference','conference_short','reference1','reference2','reference3']
sigmod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4195 entries, 0 to 4194
Data columns (total 11 columns):
id                  4195 non-null object
title               4195 non-null object
title_small         4195 non-null object
year                4195 non-null int64
date                3811 non-null object
unnamed             2408 non-null object
conference          4195 non-null object
conference_short    4195 non-null object
reference1          1420 non-null object
reference2          4195 non-null object
reference3          4195 non-null int64
dtypes: int64(2), object(9)
memory usage: 360.6+ KB


In [98]:
vldb = pd.read_csv("vldb_id.txt", delimiter='\t')
vldb.columns = ['id','title','title_small','year','date','unnamed','conference','conference_short','reference1','reference2','reference3']
vldb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4323 entries, 0 to 4322
Data columns (total 11 columns):
id                  4323 non-null object
title               4323 non-null object
title_small         4323 non-null object
year                4323 non-null int64
date                3969 non-null object
unnamed             1224 non-null object
conference          4323 non-null object
conference_short    4323 non-null object
reference1          4007 non-null object
reference2          4323 non-null object
reference3          4323 non-null int64
dtypes: int64(2), object(9)
memory usage: 371.6+ KB


In [99]:
allFiles = glob.glob("./*.txt")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,delimiter='\t')
    df.columns = ['id','title','title_small','year','date','unnamed','conference','conference_short','reference1','reference2','reference3']
    list_.append(df)
frame = pd.concat(list_)

In [100]:
print((frame))

             id                                              title  \
0      7E3FCD58  Structural trend analysis for online social ne...   
1      07803372  SQL/AA: Executing SQL on an Asymmetric Archite...   
2      7993CFAF  SCOUT: prefetching for latent structure follow...   
3      61F6E244  Probabilistic nearest neighbor queries on unce...   
4      7DE9428F  SEDA: a system for search, exploration, discov...   
5      7EB4C1D8  HadoopDB: an architectural hybrid of MapReduce...   
6      772EA93B  Efficient implementation of generalized quanti...   
7      7AEF45E2        Advanced processing for ontological queries   
8      7EC996E4      Rose : compressed, log-structured replication   
9      79B67DB3  Updatable and Evolvable Transforms for Virtual...   
10     6C6A912F  Calibrating data to sensitivity in private dat...   
11     7F201A23  Effectively indexing uncertain moving objects ...   
12     7EE41A1A  WOO: a scalable and multi-tenant platform for ...   
13     802FA533  Pre

In [101]:
tfidf__max_df: (0.25, 0.50, 0.75)
tfidf__ngram_range: ((1, 1), (1, 2), (1, 3))

In [102]:
data_x = frame[['title']].as_matrix()
print(data_x)
data_y = frame[['conference_short']].as_matrix()
print(data_y)

[['Structural trend analysis for online social networks']
 ['SQL/AA: Executing SQL on an Asymmetric Architecture']
 ['SCOUT: prefetching for latent structure following queries']
 ...
 ['On the Comprehension of Program Comprehension']
 ['Some Code Smells Have a Significant but Small Effect on Faults']
 ['Amplifying Tests to Validate Exception Handling Code: An Extended Study in the Mobile Application Domain']]
[['vldb']
 ['vldb']
 ['vldb']
 ...
 ['icse']
 ['icse']
 ['icse']]


/home/jayanth/anaconda3/envs/uwindsor/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/home/jayanth/anaconda3/envs/uwindsor/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [119]:
# split the data
stratified_split = StratifiedShuffleSplit(n_splits=2, test_size=0.33)
for train_index, test_index in stratified_split.split(data_x, data_y):
    x_train, x_test = data_x[train_index], data_x[test_index]
    y_train, y_test = data_y[train_index], data_y[test_index]
train_x = [x[0].strip() for x in x_train.tolist()]
test_x = [x[0].strip() for x in x_test.tolist()]

Naive bayes

In [120]:
pipeline_NB = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stop_words)),
    ('clf', OneVsRestClassifier(MultinomialNB(
        fit_prior=True, class_prior=None))),
])
parameters_NB = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__estimator__alpha': (1e-2, 1e-3)
}

SVM

In [121]:
pipeline_SVM = Pipeline(
    [
    ('tfidf', TfidfVectorizer(stop_words=stop_words)),
    ('clf', OneVsRestClassifier(LinearSVC()))])
parameters_SVM = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    "clf__estimator__C": [0.01, 0.1, 1],
    "clf__estimator__class_weight": ['balanced', None],
}

logistic regression

In [122]:
pipeline_logistic = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stop_words)),
    ('clf', OneVsRestClassifier(LogisticRegression(solver='sag')))])
parameters_logistic = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    "clf__estimator__C": [0.01, 0.1, 1],
    "clf__estimator__class_weight": ['balanced', None],
}

In [123]:
grid_search_tune = GridSearchCV(
    pipeline_SVM, parameters_SVM, cv=2, n_jobs=2, verbose=3)
grid_search_tune.fit(train_x, y_train)


print("Best parameters set fot SVM are:")
print(grid_search_tune.best_estimator_.steps)

# measuring performance on test set
print("Applying best classifier on test data:")
best_clf = grid_search_tune.best_estimator_
predictions = best_clf.predict(test_x)

print(classification_report(y_test, predictions))

Fitting 2 folds for each of 54 candidates, totalling 108 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    7.8s
[Parallel(n_jobs=2)]: Done 108 out of 108 | elapsed:   32.1s finished


Best parameters set fot SVM are:
[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.25, max_features=None, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words={'until', 'are', "aren't", 'when', 'which', "weren't", 'me', 'having', 'those', 'an', 'mightn', 'there', 'my', 'above', 'd', "didn't", 'not', 'wouldn', 'am', 'as', 'was', 'shan', 'now', 'o', 'very', 'these', 'the', 'through', 'ain', 'haven', 'under', 'he', 'can', 'yourselves', "shan't", '...er', 'between', 'but', 'him', 'in', "couldn't", 'aren', 'why', 'does', 'she', 'i', 'both', 'couldn'},
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)), ('clf', OneVsRestClassifier(estimator=LinearSVC(C=1, class_weight='balanced', dual=True, fit_intercept=True,
 

In [124]:
grid_search_tune = GridSearchCV(
    pipeline_NB
    , parameters_NB, cv=2, n_jobs=2, verbose=3)
grid_search_tune.fit(train_x, y_train)


print("Best parameters set:")
print(grid_search_tune.best_estimator_.steps)

# measuring performance on test set
print("Applying best classifier on test data:")
best_clf = grid_search_tune.best_estimator_
predictions = best_clf.predict(test_x)

print(classification_report(y_test, predictions))

Fitting 2 folds for each of 18 candidates, totalling 36 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    8.4s
[Parallel(n_jobs=2)]: Done  36 out of  36 | elapsed:   11.0s finished


Best parameters set:
[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.25, max_features=None, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words={'until', 'are', "aren't", 'when', 'which', "weren't", 'me', 'having', 'those', 'an', 'mightn', 'there', 'my', 'above', 'd', "didn't", 'not', 'wouldn', 'am', 'as', 'was', 'shan', 'now', 'o', 'very', 'these', 'the', 'through', 'ain', 'haven', 'under', 'he', 'can', 'yourselves', "shan't", '...er', 'between', 'but', 'him', 'in', "couldn't", 'aren', 'why', 'does', 'she', 'i', 'both', 'couldn'},
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)), ('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True),
          n_jobs=None))

In [125]:
grid_search_tune = GridSearchCV(
    pipeline_logistic
    , parameters_logistic, cv=2, n_jobs=2, verbose=3)
grid_search_tune.fit(train_x, y_train)


print("Best parameters set:")
print(grid_search_tune.best_estimator_.steps)

# measuring performance on test set
print("Applying best classifier on test data:")
best_clf = grid_search_tune.best_estimator_
predictions = best_clf.predict(test_x)

print(classification_report(y_test, predictions))

Fitting 2 folds for each of 54 candidates, totalling 108 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:   13.6s
[Parallel(n_jobs=2)]: Done 108 out of 108 | elapsed:   55.2s finished


Best parameters set:
[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words={'until', 'are', "aren't", 'when', 'which', "weren't", 'me', 'having', 'those', 'an', 'mightn', 'there', 'my', 'above', 'd', "didn't", 'not', 'wouldn', 'am', 'as', 'was', 'shan', 'now', 'o', 'very', 'these', 'the', 'through', 'ain', 'haven', 'under', 'he', 'can', 'yourselves', "shan't", '...er', 'between', 'but', 'him', 'in', "couldn't", 'aren', 'why', 'does', 'she', 'i', 'both', 'couldn'},
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)), ('clf', OneVsRestClassifier(estimator=LogisticRegression(C=1, class_weight='balanced', dual=False,
          fit_intercept=

K means clustering

In [ ]:
from sklearn.cluster import KMeans 


In [113]:
print(y_train)

[['icse']
 ['sigmod']
 ['icse']
 ...
 ['icse']
 ['sigmod']
 ['icse']]


In [114]:
print(len(y_train))

12464


In [115]:
print(len(predictions))

6139
